# Background

- The platform has 6 years worth of data.
- Problem: Issues with retention, reengagement, campaign targeting, and lack of general understanding of the user base
- Ask: How to group players based on recent behavior, general purchase behavior, and extreme purchase behavior

# Data Profile

- Business domain: scheduling platform for services
- Snapshot date: 2022-09-03
- Disclaimer: Data is private and have been masked. It will not be shared.

# Setup

In [1]:
%pip install ../.

Processing c:\users\franc\git-repo\scheduling-platform-segmentation
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scheduling-platform-segmentation: filename=scheduling_platform_segmentation-0.0.1-py3-none-any.whl size=4629 sha256=0aa500f726a11c39b95011749e9ee7e7ec88ef4b3f3f73f0acb1769d4d9970f9
  Stored in directory: c:\users\franc\appdata\local\pip\cache\wheels\0f\b0\cc\22072402b1c65edb72b7efff9548c7464e42d8420b35ab478b
Successfully built scheduling-platform-segmentation
  Attempting uninstall: scheduling-platform-segmentation
    Found existing installation: scheduling-platform-segmentation 0.0.1
    Uninstalling scheduling-platform-segmentation-0.0.1:
      Successfully uninstalled scheduling-platform-segmentation-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

from copy import deepcopy

from scheduling_platform_segmentation.constants import QUANTILE_LIST
from scheduling_platform_segmentation.preprocess.rfm import (
    calculate_weighted_recency,
    clean_weighted_recency,
    generate_quantile_dictionary,
    generate_conditions,
    generate_trad_rf_segments
)


In [3]:
pdf_raw_user_by_vendor_by_store = pd.read_csv('user_data_by_vendor_by_store.csv')
pdf_raw_user_by_vendor_by_store = pdf_raw_user_by_vendor_by_store[[c for c in pdf_raw_user_by_vendor_by_store.columns if c != 'Unnamed: 0']]
pdf_raw_user_by_vendor_by_store

,user_id,install_date,vendor_id,store_id,tenure,store_days_since_last_transaction,store_num_transactions,store_total_dollar_spend
0,142488,2022-07-23,195,269,41.737250,12.899776,2,15400.0
1,133793,2022-06-01,195,269,93.859821,20.668556,3,33000.0
2,139901,2022-07-09,195,269,55.806231,55.806223,1,33000.0
3,134030,2022-06-02,195,269,92.660766,63.704679,3,19600.0
4,140978,2022-07-14,195,269,50.750846,23.745670,3,13400.0
...,...,...,...,...,...,...,...,...
92250,110297,2021-11-29,139,219,277.861396,81.845761,2,50300.0
92251,108382,2021-11-15,139,219,291.649244,88.853760,2,59196.0
92252,123325,2022-03-22,139,219,164.875360,95.752685,1,67050.0
92253,17108,2018-08-22,146,226,1472.914944,415.731643,1,500.0


In [4]:
# pdf_raw_user_purchase_behavior = pd.read_csv('user_data_purchase_behavior.csv')
# pdf_raw_user_purchase_behavior = pdf_raw_user_purchase_behavior[[c for c in pdf_raw_user_purchase_behavior.columns if c != 'Unnamed: 0']]
# pdf_raw_user_purchase_behavior

# Recent Purchase Behavior

In [5]:
pdf_raw_user_by_vendor_by_store

,user_id,install_date,vendor_id,store_id,tenure,store_days_since_last_transaction,store_num_transactions,store_total_dollar_spend
0,142488,2022-07-23,195,269,41.737250,12.899776,2,15400.0
1,133793,2022-06-01,195,269,93.859821,20.668556,3,33000.0
2,139901,2022-07-09,195,269,55.806231,55.806223,1,33000.0
3,134030,2022-06-02,195,269,92.660766,63.704679,3,19600.0
4,140978,2022-07-14,195,269,50.750846,23.745670,3,13400.0
...,...,...,...,...,...,...,...,...
92250,110297,2021-11-29,139,219,277.861396,81.845761,2,50300.0
92251,108382,2021-11-15,139,219,291.649244,88.853760,2,59196.0
92252,123325,2022-03-22,139,219,164.875360,95.752685,1,67050.0
92253,17108,2018-08-22,146,226,1472.914944,415.731643,1,500.0


In [96]:
def calculate_weighted_recency(row):
    """this function computes the recency metric, weighted by the tenure. 
    formula is different depending on the tenure because of how the formulas
    behave in the respective ranges.

    The weighted recency metric takes into account the tenure of a user when
    considering how long ago the user last spent

    1 - row['recency'] / row['tenure'] => 1 - # of days ago over total possible days

    """

    if row['tenure'] >= 1 :
        return row['recency'] / row['tenure']
    else:
        return (1 - row['recency'] / row['tenure'])

def clean_weighted_recency(row):
    """explicitly bound the metric from 0 and 1

    """
    if row['weighted_recency'] < 0:
        return 0
    elif row['weighted_recency'] > 1:
        return 1
    else:
        return row['weighted_recency']

In [138]:
pdf_user_level = (
    pdf_raw_user_by_vendor_by_store
    .groupby('user_id')
    .agg(install_date=('install_date', 'min'),
         cnt_vendors=('vendor_id', 'nunique'),
         tenure=('tenure', 'max'),
         recency=('store_days_since_last_transaction', 'min'),
         frequency=('store_num_transactions', 'sum'),
         monetary=('store_total_dollar_spend', 'sum')
         )
    )



In [139]:
pdf_user_level

,install_date,cnt_vendors,tenure,recency,frequency,monetary
user_id,,,,,,
5,2016-03-31,1,2346.602576,70.874920,3,1495.0
52,2016-07-15,1,2240.230525,1574.828366,1,300.0
56,2016-09-01,6,2192.329370,535.607214,14,7480.0
57,2016-09-08,4,2185.122283,1742.387525,6,4098.0
73,2016-11-11,1,2121.104485,939.375335,2,755.0
...,...,...,...,...,...,...
148312,2022-09-02,1,0.724430,0.724201,1,550.0
148333,2022-09-02,1,0.619158,0.619144,1,3028.0
148347,2022-09-02,1,0.544632,0.544619,1,798.0


In [140]:
def generate_quantile_dictionary(pdf, quantile_list):
    """generate quantile dictionary for each metric. the quantile dict is going to be used to create rwfm scores

    :param pdf: _description_
    :type pdf: _type_
    :return: _description_
    :rtype: _type_
    """

    dict_quantile = {}
    quantiles = pdf[['tenure', 'recency', 'frequency', 'monetary',
                     ]].quantile(q=quantile_list)
    dict_quantile[f'tenure'] = dict(quantiles[f'tenure'])
    dict_quantile[f'recency'] = dict(quantiles[f'recency'])
    dict_quantile[f'frequency'] = dict(quantiles[f'frequency'])
    dict_quantile[f'monetary'] = dict(quantiles[f'monetary'])
    return dict_quantile

In [141]:
QUANTILE_LIST = [0.2, 0.4, 0.6, 0.8]

In [142]:
dict_quantile = generate_quantile_dictionary(pdf_user_level, QUANTILE_LIST) # save this into database
dict_quantile

{'tenure': {0.2: 307.80049852,
  0.4: 681.7994606200001,
  0.6: 954.70483208,
  0.8: 1273.7553182000001},
 'recency': {0.2: 190.58063782000002,
  0.4: 625.7738795,
  0.6: 795.4346637599999,
  0.8: 1124.701215},
 'frequency': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 2.0},
 'monetary': {0.2: 230.0, 0.4: 450.0, 0.6: 795.0, 0.8: 1745.0}}

In [144]:
pdf_rfm_scored = deepcopy(pdf_user_level)
for metric in dict_quantile.keys():
    if metric in ('recency'):
        reverse = 1
    else:
        reverse = 0

    conditions, values = generate_conditions(pdf_rfm_scored, dict_quantile, metric, reverse=reverse)
    pdf_rfm_scored[metric[0]] = np.select(conditions, values)

pdf_rfm_scored

,install_date,cnt_vendors,tenure,recency,frequency,monetary,t,r,f,m
user_id,,,,,,,,,,
5,2016-03-31,1,2346.602576,70.874920,3,1495.0,5,5,5,4
52,2016-07-15,1,2240.230525,1574.828366,1,300.0,5,1,1,2
56,2016-09-01,6,2192.329370,535.607214,14,7480.0,5,4,5,5
57,2016-09-08,4,2185.122283,1742.387525,6,4098.0,5,1,5,5
73,2016-11-11,1,2121.104485,939.375335,2,755.0,5,2,4,3
...,...,...,...,...,...,...,...,...,...,...
148312,2022-09-02,1,0.724430,0.724201,1,550.0,1,5,1,3
148333,2022-09-02,1,0.619158,0.619144,1,3028.0,1,5,1,5
148347,2022-09-02,1,0.544632,0.544619,1,798.0,1,5,1,4


,install_date,cnt_vendors,tenure,recency,frequency,monetary,weighted_recency,r,w,f,m
user_id,,,,,,,,,,,
3017,2017-03-17,1,1995.208448,1995.208311,1,360.0,4.714814e-15,1,2,1,2
3018,2017-03-17,1,1995.166015,1995.165839,1,360.0,7.781571e-15,1,2,1,2
3351,2017-03-28,1,1984.406480,1984.406425,1,360.0,7.681820e-16,1,2,1,2
4210,2017-05-05,1,1946.223796,1946.223700,1,360.0,2.433083e-15,1,2,1,2
4302,2017-05-10,1,1941.345314,1941.345270,1,700.0,5.136884e-16,1,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...
137405,2022-06-24,1,70.896102,70.896096,1,450.0,7.574006e-15,5,2,1,2
137417,2022-06-24,1,70.877338,70.877332,1,550.0,8.697373e-15,5,2,1,3
137432,2022-06-24,1,70.849563,70.849558,1,450.0,5.895538e-15,5,2,1,2


In [ ]:
def generate_trad_rf_segments(pdf, log=0, weighted=0):
    """tag each user with traditional RF segments based on RWFM data

    """
    pdf_copy = deepcopy(pdf)
    if log == 0:
        sfx = '_log'
    else:
        sfx = ''

    if weighted == 0:
        recency = f'r{sfx}'
    else:
        recency = f'w{sfx}'

    conditions = [
        (pdf_copy[recency] >= 1) & (pdf_copy[recency] <= 2) & (pdf_copy[f'f{sfx}'] >= 1) & (pdf_copy[f'f{sfx}'] <= 2), # hibernating
        (pdf_copy[recency] >= 1) & (pdf_copy[recency] <= 2) & (pdf_copy[f'f{sfx}'] >= 3) & (pdf_copy[f'f{sfx}'] <= 4), # at_risk
        (pdf_copy[recency] >= 1) & (pdf_copy[recency] <= 2) & (pdf_copy[f'f{sfx}'] == 5), # cant_lose
        (pdf_copy[recency] == 3) & (pdf_copy[f'f{sfx}'] >= 1) & (pdf_copy[f'f{sfx}'] <= 2), # about_to_sleep
        (pdf_copy[recency] == 3) & (pdf_copy[f'f{sfx}'] == 3), # need_attention
        (pdf_copy[recency] >= 3) & (pdf_copy[recency] <= 4) & (pdf_copy[f'f{sfx}'] >= 4) & (pdf_copy[f'f{sfx}'] <= 5), # loyal_customers
        (pdf_copy[recency] == 4) & (pdf_copy[f'f{sfx}'] == 1), # promising
        (pdf_copy[recency] == 5) & (pdf_copy[f'f{sfx}'] == 1), # new_customers
        (pdf_copy[recency] >= 4) & (pdf_copy[recency] <= 5) & (pdf_copy[f'f{sfx}'] >= 2) & (pdf_copy[f'f{sfx}'] <= 3), # potential_loyalists
        (pdf_copy[recency] == 5) & (pdf_copy[f'f{sfx}'] >= 4) & (pdf_copy[f'f{sfx}'] <= 5), # champions
    ]

    values = TRAD_RF_SEGMENTS_LIST
    labels = np.select(conditions, values)

    return labels

In [10]:
weighted_labels = generate_trad_rf_segments(pdf_rfm, weighted=1)
normal_labels = generate_trad_rf_segments(pdf_rfm, weighted=0)

In [11]:
pdf_rfm_labeled = deepcopy(pdf_rfm)
pdf_rfm_labeled['rfm'] = normal_labels
pdf_rfm_labeled['wfm'] = weighted_labels

In [12]:
pdf_rfm_labeled

,install_date,cnt_vendors,tenure,recency,frequency,monetary,weighted_recency,r,w,f,m,rfm,wfm
user_id,,,,,,,,,,,,,
5,2016-03-31,1,2346.602576,70.874920,3,1495.0,0.940506,5,5,5,4,champions,champions
52,2016-07-15,1,2240.230525,1574.828366,1,300.0,0.088223,1,5,1,2,hibernating,new_customers
56,2016-09-01,6,2192.329370,535.607214,14,7480.0,0.571068,4,5,5,5,loyal_customers,champions
57,2016-09-08,4,2185.122283,1742.387525,6,4098.0,0.041052,1,5,5,5,cant_lose,champions
73,2016-11-11,1,2121.104485,939.375335,2,755.0,0.310393,2,5,4,3,at_risk,champions
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148312,2022-09-02,1,0.724430,0.724201,1,550.0,0.999684,5,5,1,3,new_customers,new_customers
148333,2022-09-02,1,0.619158,0.619144,1,3028.0,0.999978,5,5,1,5,new_customers,new_customers
148347,2022-09-02,1,0.544632,0.544619,1,798.0,0.999977,5,5,1,4,new_customers,new_customers
